In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

df=pd.read_csv(r'D:\Desktop\Regression\deliverytime.csv')
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  int64  
 3   Delivery_person_Ratings      45593 non-null  float64
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Type_of_order                45593 non-null  object 
 9   Type_of_vehicle              45593 non-null  object 
 10  Time_taken(min)              45593 non-null  int64  
dtypes: float64(5), int64(2), object(4)
memory usage: 3.8+ MB


In [4]:
df.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Type_of_order                  0
Type_of_vehicle                0
Time_taken(min)                0
dtype: int64

In [5]:
R=6371
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c

In [6]:
df['distance']=np.nan
for i in range(len(df)):
    df.loc[i,'distance']=distcalculate(df.loc[i,'Restaurant_latitude'],df.loc[i,'Restaurant_longitude'],df.loc[i,'Delivery_location_latitude'],
                                       df.loc[i,'Delivery_location_longitude'])

In [7]:
df['distance']

0         3.025149
1        20.183530
2         1.552758
3         7.790401
4         6.210138
           ...    
45588     1.489846
45589    11.007735
45590     4.657195
45591     6.232393
45592    12.074396
Name: distance, Length: 45593, dtype: float64

In [8]:
figure=px.scatter(data_frame=df,x='distance',y='Time_taken(min)',size='Time_taken(min)',trendline='ols',title='TimeTaken Vs Distance')
figure.show()

In [9]:
figure = px.scatter(data_frame = df, 
                    x="Delivery_person_Age",
                    y="Time_taken(min)", 
                    size="Time_taken(min)", 
                    color = "distance",
                    trendline="ols", 
                    title = "Relationship Between Time Taken and Age")
figure.show()

In [10]:
%pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from sklearn.model_selection import train_test_split
x=df[['Delivery_person_Age','Delivery_person_Ratings','distance']]
y=df['Time_taken(min)']

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=45)
x_train

,Delivery_person_Age,Delivery_person_Ratings,distance
12131,26,4.9,16.903142
17419,32,4.7,13.969878
36138,33,4.8,9.314885
42775,32,4.7,17.297866
8927,29,4.9,9.313348
...,...,...,...
32469,35,4.8,9.192572
16317,30,4.6,19.618739
12895,34,4.7,4.469027
6012,35,5.0,7.862668


In [13]:
%pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
x_train.shape

(36474, 3)

In [15]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

c:\Users\91763\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 3, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(x_train,y_train,batch_size=1,epochs=10)

Epoch 1/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 584s 16ms/step - loss: 77.6839
Epoch 2/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 552s 14ms/step - loss: 64.8749
Epoch 3/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 510s 14ms/step - loss: 62.0410
Epoch 4/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 241s 7ms/step - loss: 61.5222
Epoch 5/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 152s 4ms/step - loss: 60.1108
Epoch 6/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 154s 4ms/step - loss: 59.5056
Epoch 7/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 559s 14ms/step - loss: 59.5385
Epoch 8/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 533s 13ms/step - loss: 58.7854
Epoch 9/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 494s 14ms/step - loss: 59.7992
Epoch 10/10
36474/36474 ━━━━━━━━━━━━━━━━━━━━ 660s 18ms/step - loss: 58.6132


In [17]:
%pip install pydot

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
